# BiLSTM+CRF模型训练，使用word2vec词嵌入

In [1]:
# 显卡查看
! nvidia-smi

Thu May 21 00:57:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# 依赖安装
! pip install fastNLP

     |████████████████████████████████| 276kB 8.1MB/s 
     |████████████████████████████████| 1.4MB 17.1MB/s 
  Created wheel for fastNLP: filename=FastNLP-0.5.5-cp36-none-any.whl size=332757 sha256=a9d2cd1709a1ba3796be2693a931dceaaf6ed980877461ad3c7ac790bc4b8db1
  Stored in directory: /root/.cache/pip/wheels/13/b9/4e/7e7b9c2e3deae523b2eec14157ed112ce09bf1dee5483a48ae
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434676 sha256=50d024733e9c82d17d8dbdd5b8e783d8392a2072a482fbb857e26e6e7c2120c1
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built fastNLP nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


加载数据集

In [3]:
import sys

from fastNLP.core import Const
from fastNLP.io import PeopleDailyNERLoader
from fastNLP.io import PeopleDailyPipe

sys.path.insert(0, '/content/drive/My Drive/my_framework/qyt_clue/')  # 定义搜索路径的优先顺序，序号从0开始，表示最大优先级

import myClue  # noqa
print('myClue module path :{}'.format(myClue.__file__))  # 输出测试模块文件位置
from myClue.core import logger  # noqa
from myClue.core.utils import print_data_bundle  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa

myClue module path :/content/drive/My Drive/my_framework/qyt_clue/myClue/__init__.py


In [0]:
# 原始语料
# train_file_config = {
#     'train': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train.conll',
#     'dev': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/dev.conll',
#     'test': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/test.conll',
# }

In [0]:
# 使用最佳模型预测后的语料
train_file_config = {
    'train': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train_bert_predict.conll',
    'dev': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/dev_bert_predict.conll',
    'test': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/test.conll',
}

In [6]:
# train_data_bundle_pkl_file = './data/weibo_NER/train_data_bundle.pkl'
logger.info('数据加载')
data_loader = PeopleDailyNERLoader()
data_bundle = data_loader.load(train_file_config)
print_data_bundle(data_bundle)
logger.info('数据预处理')
data_pipe = PeopleDailyPipe()
data_bundle = data_pipe.process(data_bundle)
data_bundle.rename_field(field_name=Const.CHAR_INPUT, new_field_name=Const.INPUT, ignore_miss_dataset=True, rename_vocab=True)
print_data_bundle(data_bundle)
# save_serialize_obj(data_bundle, train_data_bundle_pkl_file)
# logger.info('数据预处理后进行序列化：{}'.format(train_data_bundle_pkl_file))

2020-05-21 00:58:03 I [<ipython-input-6-a24be1271d63>:2] 数据加载
2020-05-21 00:58:06 I [utils.py:16] dataset name : train
2020-05-21 00:58:06 I [utils.py:17] dataset len : 1350
2020-05-21 00:58:06 I [utils.py:18] dataset example : 
2020-05-21 00:58:06 I [utils.py:19] 
+------------------------------------------+------------------------------------------+
| raw_chars                                | target                                   |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['浑', '身', '酸', '疼', '，', '两', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+----------------

+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+


2020-05-21 00:58:07 I [utils.py:16] dataset name : train
2020-05-21 00:58:07 I [utils.py:17] dataset len : 1350
2020-05-21 00:58:07 I [utils.py:18] dataset example : 
2020-05-21 00:58:07 I [utils.py:19] 
+------------------------+------------------------+---------+------------------------+
| raw_chars              | target                 | seq_len | words                  |
+------------------------+------------------------+---------+------------------------+
| ['科', '技', '全', ... | [0, 0, 0, 0, 0, 0, ... | 26      | [792, 1015, 156, 19... |
| ['对', '，', '输', ... | [0, 0, 0, 0, 0, 0, ... | 15      | [123, 2, 1205, 115,... |
| ['今', '天', '下', ... | [0, 0, 0, 0, 0, 0, ... | 79      | [164, 33, 65, 739, ... |
| ['今', '年', '拜', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [164, 92, 880, 92, ... |
| ['浑', '身', '酸', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [2072, 221, 1056, 9... |
+------------------------+------------------------+---------+------------------------+
2020-05-21 00:58:07 I [utils

+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+---------

模型训练

In [0]:
import os
import sys
import time

import torch

from fastNLP import Const
from fastNLP.embeddings import StaticEmbedding
from fastNLP.models import BiLSTMCRF
from fastNLP import SpanFPreRecMetric
from fastNLP import Trainer
from fastNLP import LossInForward
from torch.optim import Adam
from fastNLP import Tester

from myClue.core import logger  # noqa
from myClue.core.callback import EarlyStopCallback  # noqa
from myClue.tools.serialize import load_serialize_obj  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.tools.file import init_file_path  # noqa

In [8]:
# train_data_bundle_pkl_file = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train_data_bundle.pkl'
model_path = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bilstm_crf'
init_file_path(model_path)
logger.add_file_handler(os.path.join(model_path, 'log_{}.txt'.format(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()))))  # 日志写入文件
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
# logger.warn('加载数据集')
# data_bundle = load_serialize_obj(train_data_bundle_pkl_file)
logger.warn('获取词典')
char_vocab = data_bundle.get_vocab('words')
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = data_bundle.get_vocab('target')
logger.info('target_vocab:{}'.format(target_vocab))
save_serialize_obj(char_vocab, char_vocab_pkl_file)
save_serialize_obj(target_vocab, target_vocab_pkl_file)
logger.info('词典序列化:{}'.format(char_vocab_pkl_file))
logger.warn('选择预训练词向量')
word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name='/content/drive/My Drive/data/cn_char_fastnlp_100d/cn_char_fastnlp_100d.txt')
logger.warn('神经网络模型')
model = BiLSTMCRF(embed=word2vec_embed, num_classes=len(target_vocab), num_layers=1, hidden_size=200, dropout=0.5,
                  target_vocab=target_vocab)
logger.info(model)
logger.warn('训练超参数设定')
loss = LossInForward()
optimizer = Adam([param for param in model.parameters() if param.requires_grad])
# metric = AccuracyMetric()
metric = SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab(Const.TARGET), only_gross=False)  # 若only_gross=False, 即还会返回各个label的metric统计值
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
logger.info('device:{}'.format(device))

2020-05-21 00:58:09 W [<ipython-input-8-480b2396853e>:9] 获取词典
2020-05-21 00:58:09 I [<ipython-input-8-480b2396853e>:11] char_vocab:Vocabulary(['科', '技', '全', '方', '位']...)
2020-05-21 00:58:09 I [<ipython-input-8-480b2396853e>:13] target_vocab:Vocabulary(['O', 'B-PER.NOM', 'I-PER.NOM', 'B-LOC.NAM', 'I-LOC.NAM']...)
2020-05-21 00:58:11 I [<ipython-input-8-480b2396853e>:16] 词典序列化:/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bilstm_crf/vocab_char.pkl
2020-05-21 00:58:11 W [<ipython-input-8-480b2396853e>:17] 选择预训练词向量
/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
2020-05-21 00:58:16 W [<ipython-input-8-480b2396853e>:19] 神经网络模型

Found 3321 out of 3471 words in the pre-training embedding.


In [9]:
# 模型训练
batch_size = 32
n_epochs = 200
early_stopping = 10
trainer = Trainer(
    save_path=model_path,
    train_data=data_bundle.get_dataset('train'),
    model=model,
    loss=loss,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epochs=n_epochs,
    dev_data=data_bundle.get_dataset('dev'),
    metrics=metric,
    metric_key='f',
    device=device,
    callbacks=[EarlyStopCallback(early_stopping)])
logger.warn('开始训练')
trainer.train()

input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 



2020-05-21 00:58:26 W [<ipython-input-9-d54362d8f065>:18] 开始训练


training epochs started 2020-05-21-00-58-26-403207


2020-05-21 00:58:31 W [callback.py:38] ======epoch : 1 , early stopping : 0/10======
2020-05-21 00:58:31 I [callback.py:40] metric_key : f, metric_value : 0.0
2020-05-21 00:58:31 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-per.nom': 0.0, 'pre-per.nom': 0.0, 'rec-per.nom': 0.0, 'f-loc.nom': 0.0, 'pre-loc.nom': 0.0, 'rec-loc.nom': 0.0, 'f-per.nam': 0.0, 'pre-per.nam': 0.0, 'rec-per.nam': 0.0, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-gpe.nam': 0.0, 'pre-gpe.nam': 0.0, 'rec-gpe.nam': 0.0, 'f-org.nam': 0.0, 'pre-org.nam': 0.0, 'rec-org.nam': 0.0, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f': 0.0, 'pre': 0.0, 'rec': 0.0}}


Evaluate data in 0.35 seconds!
Evaluation on dev at Epoch 1/200. Step:43/8600: 
SpanFPreRecMetric: f-per.nom=0.0, pre-per.nom=0.0, rec-per.nom=0.0, f-loc.nom=0.0, pre-loc.nom=0.0, rec-loc.nom=0.0, f-per.nam=0.0, pre-per.nam=0.0, rec-per.nam=0.0, f-loc.nam=0.0, pre-loc.nam=0.0, rec-loc.nam=0.0, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-gpe.nam=0.0, pre-gpe.nam=0.0, rec-gpe.nam=0.0, f-org.nam=0.0, pre-org.nam=0.0, rec-org.nam=0.0, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f=0.0, pre=0.0, rec=0.0



...

2020-05-21 01:00:50 W [callback.py:38] ======epoch : 32 , early stopping : 9/10======
2020-05-21 01:00:50 I [callback.py:40] metric_key : f, metric_value : 0.48048
2020-05-21 01:00:50 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-per.nom': 0.604651, 'pre-per.nom': 0.8125, 'rec-per.nom': 0.481481, 'f-loc.nom': 0.111111, 'pre-loc.nom': 1.0, 'rec-loc.nom': 0.058824, 'f-per.nam': 0.502994, 'pre-per.nam': 0.531646, 'rec-per.nam': 0.477273, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-gpe.nam': 0.318182, 'pre-gpe.nam': 0.7, 'rec-gpe.nam': 0.205882, 'f-org.nam': 0.169014, 'pre-org.nam': 0.4, 'rec-org.nam': 0.107143, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f': 0.48048, 'pre': 0.683761, 'rec': 0.37037}}


Evaluate data in 0.35 seconds!
Evaluation on dev at Epoch 32/200. Step:1376/8600: 
SpanFPreRecMetric: f-per.nom=0.604651, pre-per.nom=0.8125, rec-per.nom=0.481481, f-loc.nom=0.111111, pre-loc.nom=1.0, rec-loc.nom=0.058824, f-per.nam=0.502994, pre-per.nam=0.531646, rec-per.nam=0.477273, f-loc.nam=0.0, pre-loc.nam=0.0, rec-loc.nam=0.0, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-gpe.nam=0.318182, pre-gpe.nam=0.7, rec-gpe.nam=0.205882, f-org.nam=0.169014, pre-org.nam=0.4, rec-org.nam=0.107143, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f=0.48048, pre=0.683761, rec=0.37037



2020-05-21 01:00:55 W [callback.py:38] ======epoch : 33 , early stopping : 10/10======
2020-05-21 01:00:55 I [callback.py:40] metric_key : f, metric_value : 0.519637
2020-05-21 01:00:55 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-per.nom': 0.647727, 'pre-per.nom': 0.838235, 'rec-per.nom': 0.527778, 'f-loc.nom': 0.111111, 'pre-loc.nom': 1.0, 'rec-loc.nom': 0.058824, 'f-per.nam': 0.516556, 'pre-per.nam': 0.619048, 'rec-per.nam': 0.443182, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-gpe.nam': 0.382979, 'pre-gpe.nam': 0.692308, 'rec-gpe.nam': 0.264706, 'f-org.nam': 0.25, 'pre-org.nam': 0.5625, 'rec-org.nam': 0.160714, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f': 0.519637, 'pre': 0.747826, 'rec': 0.398148}}
2020-05-21 01:00:55 I [callback.py:46] reach early stopping patience, stop training.
2020-05-21 01:00:55 I [callback.py:55] Early Stopping triggered in epoch 33!


Evaluate data in 0.34 seconds!
Reloaded the best model.

In Epoch:22/Step:946, got best dev performance:
SpanFPreRecMetric: f-per.nom=0.68617, pre-per.nom=0.80625, rec-per.nom=0.597222, f-loc.nom=0.0, pre-loc.nom=0.0, rec-loc.nom=0.0, f-per.nam=0.547771, pre-per.nam=0.623188, rec-per.nam=0.488636, f-loc.nam=0.0, pre-loc.nam=0.0, rec-loc.nam=0.0, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-gpe.nam=0.272727, pre-gpe.nam=0.6, rec-gpe.nam=0.176471, f-org.nam=0.184211, pre-org.nam=0.35, rec-org.nam=0.125, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f=0.535456, pre=0.714286, rec=0.428241


{'best_epoch': 22,
 'best_eval': {'SpanFPreRecMetric': {'f': 0.535456,
   'f-gpe.nam': 0.272727,
   'f-gpe.nom': 0.0,
   'f-loc.nam': 0.0,
   'f-loc.nom': 0.0,
   'f-org.nam': 0.184211,
   'f-org.nom': 0.0,
   'f-per.nam': 0.547771,
   'f-per.nom': 0.68617,
   'pre': 0.714286,
   'pre-gpe.nam': 0.6,
   'pre-gpe.nom': 0.0,
   'pre-loc.nam': 0.0,
   'pre-loc.nom': 0.0,
   'pre-org.nam': 0.35,
   'pre-org.nom': 0.0,
   'pre-per.nam': 0.623188,
   'pre-per.nom': 0.80625,
   'rec': 0.428241,
   'rec-gpe.nam': 0.176471,
   'rec-gpe.nom': 0.0,
   'rec-loc.nam': 0.0,
   'rec-loc.nom': 0.0,
   'rec-org.nam': 0.125,
   'rec-org.nom': 0.0,
   'rec-per.nam': 0.488636,
   'rec-per.nom': 0.597222}},
 'best_step': 946,
 'seconds': 148.95}

In [10]:
# 评测
logger.warn('训练后评估')
tester = Tester(data=data_bundle.get_dataset('test'), model=model, metrics=metric, batch_size=64, device=device)
test_result = tester.test()
logger.info('评估结果：\n{}'.format(test_result))

2020-05-21 01:00:55 W [<ipython-input-10-2a8ad2ebd772>:2] 训练后评估


2020-05-21 01:00:55 I [<ipython-input-10-2a8ad2ebd772>:5] 评估结果：
{'SpanFPreRecMetric': {'f-per.nom': 0.54, 'pre-per.nom': 0.632812, 'rec-per.nom': 0.47093, 'f-loc.nom': 0.166667, 'pre-loc.nom': 0.333333, 'rec-loc.nom': 0.111111, 'f-per.nam': 0.558659, 'pre-per.nam': 0.757576, 'rec-per.nam': 0.442478, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-gpe.nam': 0.507042, 'pre-gpe.nam': 0.75, 'rec-gpe.nam': 0.382979, 'f-org.nam': 0.111111, 'pre-org.nam': 0.2, 'rec-org.nam': 0.076923, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f': 0.46789, 'pre': 0.648305, 'rec': 0.366029}}


Evaluate data in 0.23 seconds!
[tester] 
SpanFPreRecMetric: f-per.nom=0.54, pre-per.nom=0.632812, rec-per.nom=0.47093, f-loc.nom=0.166667, pre-loc.nom=0.333333, rec-loc.nom=0.111111, f-per.nam=0.558659, pre-per.nam=0.757576, rec-per.nam=0.442478, f-loc.nam=0.0, pre-loc.nam=0.0, rec-loc.nam=0.0, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-gpe.nam=0.507042, pre-gpe.nam=0.75, rec-gpe.nam=0.382979, f-org.nam=0.111111, pre-org.nam=0.2, rec-org.nam=0.076923, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f=0.46789, pre=0.648305, rec=0.366029
